# Эвотор
## Настройки

In [ ]:
from wb_tools import *

logging.getLogger().setLevel(logging.INFO)

## Обновление
### Исходник из базы

In [ ]:
# Выгрузка схемы базы из Эвотора
'''
select col.column_id, 
       col.owner as schema_name,
       col.table_name, 
       col.column_name, 
       col.data_type, 
       col.data_length, 
       col.data_precision, 
       col.data_scale, 
       col.nullable
from sys.all_tab_columns col
inner join sys.all_tables t on col.owner = t.owner 
                              and col.table_name = t.table_name
-- excluding some Oracle maintained schemas
where col.owner not in ('ANONYMOUS','CTXSYS','DBSNMP','EXFSYS', 'LBACSYS', 
   'MDSYS', 'MGMT_VIEW','OLAPSYS','OWBSYS','ORDPLUGINS', 'ORDSYS','OUTLN', 
   'SI_INFORMTN_SCHEMA','SYS','SYSMAN','SYSTEM','TSMSYS','WK_TEST','WKSYS', 
   'WKPROXY','WMSYS','XDB','APEX_040000', 'APEX_PUBLIC_USER','DIP', 
   'FLOWS_30000','FLOWS_FILES','MDDATA', 'ORACLE_OCM', 'XS$NULL',
   'SPATIAL_CSW_ADMIN_USR', 'SPATIAL_WFS_ADMIN_USR', 'PUBLIC')  
order by col.owner, col.table_name, col.column_id;
'''

df_e = pd.read_csv('./evotor_schemas.csv')


# Возьмём самые ходовые схемы для тестов 
schema_list = [
    'AIRFLOW', 
    'BIGDATA_LOADER', 
    'EVOTOR_ANALYTICS', 
    'EVOTOR_BIGDATA', 
    'EVOTOR_MARKET_REPL',
    'EVOTOR_REPORTS',
    'EVOTOR_CRM'
]

df_input = pd.read_csv('./evotor_schemas.csv')
df_input = df_input[df_input.SCHEMA.isin(schema_list)]

# Логин
login_instance = wbi_login.Login(user=WIKIBASE_LOGIN, pwd=WIKIBASE_PASSWORD)   

### Справочник параметров

In [ ]:
PROPERTY_DICT = {
    'GLOBAL': {
        'global_statements_items': gen_prop_dict(['located_in', 'instance of', 'in_company']),
        'global_references': gen_prop_dict(['Source']),
        'global_company': get_items_by_label(['Evotor Company'], item_type = 'Q').at[0, 'item']
    },
    
    # Каким параметром P будет являться этот объект у родителя?
#     'P': get_items_by_label(['COMPANY'], item_type = 'P').at[0, 'item'],
    # Квалифаеры параметра Р у родительского объекта
    'core_qualifiers': {},
    'custom_qualifiers': {},
    'statements': gen_prop_dict(['DESCRIPTION']),
    'instance_of': get_items_by_label(['Evotor Company'], item_type = 'Q').at[0, 'item'],
    'DATABASE': {
        'P': get_items_by_label(['DATABASE'], item_type = 'P').at[0, 'item'],
        'core_qualifiers': {},
        'custom_qualifiers': {},
        'statements': gen_prop_dict(['DESCRIPTION']),
        'instance_of': get_items_by_label(['Database'], item_type = 'Q').at[0, 'item'],
        'SCHEMA': {
            'P': get_items_by_label(['SCHEMA'], item_type = 'P').at[0, 'item'],
            'core_qualifiers': {},
            'custom_qualifiers': {},
            'statements': gen_prop_dict(['DESCRIPTION']),
            'instance_of': get_items_by_label(['Schema'], item_type = 'Q').at[0, 'item'],
            'TABLE': {
                'P': get_items_by_label(['TABLE'], item_type = 'P').at[0, 'item'],
                'core_qualifiers': {},
                'custom_qualifiers': {},
                'statements': gen_prop_dict(['DESCRIPTION']),
                'instance_of': get_items_by_label(['Table'], item_type = 'Q').at[0, 'item'],
                'COLUMN': {
                    'P': get_items_by_label(['COLUMN'], item_type = 'P').at[0, 'item'],
                    'core_qualifiers': {},
                    'custom_qualifiers': {},
                    'statements': gen_prop_dict(['DATA_TYPE', 'DATA_LENGTH', 'DESCRIPTION']),
                    'instance_of': get_items_by_label(['Column'], item_type = 'Q').at[0, 'item']
                }   
            }   
        }        
    }
}

### Взаимодействие с Wikibase

#### Создать компанию

In [ ]:
wcmp = WikiCompany('Evotor Company', PROPERTY_DICT, login_instance, 'Evotor Company')
wcmp.push_to_wiki()

#### Создать базы

In [ ]:
wd = WikiDatabase('dwh', PROPERTY_DICT, login_instance, 'Evotor Company')
wd.push_to_wiki()

#### Создать схемы

In [ ]:
df_input_s = df_input[['SCHEMA']].drop_duplicates()
df_input_s['DATABASE'] = 'dwh'
df_input_s['SCHEMA_LABEL'] = df_input_s['DATABASE'] + '.' + df_input_s['SCHEMA']

In [ ]:
for database_i in ['dwh']:
    for schema_i in df_input['SCHEMA'].unique():
        label_i = '.'.join([database_i, schema_i])

        ws = WikiSchema(label_i, PROPERTY_DICT, login_instance)
        ws.push_to_wiki()
        
    wd = WikiDatabase('dwh', PROPERTY_DICT, login_instance, 'Evotor Company')
    wd.push_to_wiki()

#### Создать таблицы

In [ ]:
for database_i in ['dwh']:
    for schema_i in df_input['SCHEMA'].unique():
        schema_label_i = '.'.join([database_i, schema_i])
        for table_i in df_input[df_input.SCHEMA == schema_i]['TABLE'].unique():
            table_label_i = '.'.join([database_i, schema_i, table_i])
            wt = WikiTable(table_label_i, PROPERTY_DICT, login_instance)
            wt.push_to_wiki()
            
        ws = WikiSchema(schema_label_i, PROPERTY_DICT, login_instance)
        ws.push_to_wiki()

#### Создать колонки

In [ ]:
for database_i in ['dwh']:
    for schema_i in ['BIGDATA_LOADER']: #df_input['SCHEMA'].unique():
        for table_i in df_input[df_input.SCHEMA == schema_i]['TABLE'].unique():
            table_label_i = '.'.join([database_i, schema_i, table_i])
            
            for column_i in df_input[(df_input.SCHEMA == schema_i) & (df_input.TABLE == table_i)]['COLUMN'].unique():
                column_label_i = '.'.join([database_i, schema_i, table_i, column_i])
                
                df_i = df_input[(df_input.SCHEMA == schema_i) & (df_input.TABLE == table_i) & (df_input.COLUMN == column_i)].reset_index()
                df_input_i = pd.DataFrame({
                    'STATEMENT_LABEL': ['DATA_TYPE', 'DATA_LENGTH'], 
                    'STATEMENT_VALUE': [df_i.at[0, 'DATA_TYPE'], df_i.at[0, 'DATA_LENGTH']],
                    'STATEMENT_TYPE': 'string'                    
                })
                                         
                wc = WikiColumn(column_label_i, PROPERTY_DICT, login_instance, df_input_i)
                wc.push_to_wiki()
            wt = WikiTable(table_label_i, PROPERTY_DICT, login_instance)
            wt.push_to_wiki()